Purpose of this notebook:

1. Enable quick manual segmentation of large TIF images containing multiple tiles in x-y and multiple z-planes, in a single channel
2. Create a table of cells X ROIs also containing information about the location of the cell in vivo (within a particular FOV)
3. Allow each cell to have ROIs in multiple z planes
4. Work with single image stitched in x-y if possible
5. Use ROIs from one channel to calculate intensity in other channels and produce histogram

# Setup

In [9]:
!conda install ipykernel --name py365

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.4.6
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base conda




In [10]:
!python -m ipykernel install

Installed kernelspec python3 in C:\ProgramData\jupyter\kernels\python3


In [7]:
%gui qt5 

In [27]:
# Imports in alphabetical order of module name

from PIL import Image
from keybind import KeyBinder
from prompt_toolkit.key_binding import KeyBindings
import napari
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt


# Load images

In [3]:
## Specify images to load

folder = 'Z:\HCR\9.2_FISH\S4_dapi_546_488_647-02'
base_filename = 'S4_dapi_546_488_647-02_s1z'
plane_nos = range(15, 29)
n = len(plane_nos)
print('Number of planes: {0}'.format(n))

Number of planes: 14


In [4]:
# Create 4D array to store images
img = Image.open('{0}\{1}{2}c1+2+3+4.tif'.format(folder, base_filename, str(plane_nos[0])))
h = img.height
w = img.width
c = np.array(img).shape[2]
im_array = np.zeros([n, h, w, c])
print('Size of image in pixels: {0} X {1} X {2}'.format(n, h, w))

for i in range(n):
    p = plane_nos[i]
    img = Image.open('{0}\{1}{2}c1+2+3+4.tif'.format(folder, base_filename, str(p)))
    try:
        im_array[i, :, :, :] = np.array(img)
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array = np.delete(im_array, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue


Size of image in pixels: 14 X 1896 X 3740


# Annotation

## Data structure to store information about cells

In [5]:
class Cell:
    n_cells = 1
    
    def __init__(self, cell_no):
        
        self.cell_no = cell_no
        self.z_planes = []
        self.masks = {}
        self.c_vals = np.zeros(c)
        print('Cell #{0} created'.format(Cell.n_cells))
        Cell.n_cells += 1
        
    def add_mask(self, cell_no, z_plane, mask):
        if not cell_no == self.cell_no:
            print('Trying to add mask to wrong cell!!')
            return
        else:
            self.z_planes.append(z_plane)
            self.masks['z_plane'] = mask
            
    def save_cell(self):
        cell_data[self.cell_no] = {'cell_id': self.cell_no,
                                 'z_planes': self.z_planes,
                                 'masks': self.masks,
                                 'no_planes': len(self.z_planes),
                                 'c_vals':self.c_vals}
    
    def del_cell(self):
        Cell.n_cells -= 1

In [6]:
save_loc = 'Z:\HCR\9.2_FISH'
save_file = 'S4_data.pkl'
cell_data = {i:{'cell_id': i,
                'z_planes': [],
                'masks': {},
                'no_planes': 0,
                'c_vals': np.zeros(c)} for i in range(Cell.n_cells)}

## Render image in napari GUI

In [8]:
# Open image in napari viewer - each channel as a separate layer 
viewer = napari.Viewer()

for i in range(c):
    viewer.add_image(im_array[:, :, :, i], name = 'Channel {0}'.format(i + 1), colormap = 'viridis', 
                     interpolation = 'nearest')
    
seg_channel = 1
viewer.add_shapes([], name = 'masks', opacity = 0.2, face_color = 'white', edge_color = 'red', edge_width = 3)

# Keybinding functions
@viewer.bind_key('p')
def print_names(viewer):
    print([layer.name for layer in viewer.layers])

@viewer.bind_key('n')
def new_cell(viewer):
    global cell1 
    cell1 = Cell(Cell.n_cells)

@viewer.bind_key('m')
def add_mask(viewer):
    seg_layer = viewer.layers[seg_channel]
    z_plane = seg_layer.coordinates[0]
    mask = viewer.layers['masks'].data[-1]
    cell1.add_mask(Cell.n_cells, z_plane, mask)
    
@viewer.bind_key('s')
def save_cell(viewer):
    cell1.save_cell()

C:\Users\singha\AppData\Local\Continuum\anaconda3\envs\py365\lib\site-packages\vispy\visuals\markers.py:564: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('a_edgewidth', np.float32, 1)])


## Keybindings

In [29]:
class Keybind:
    def __init__(self):
        print('object created')
        self.__ID3 = self.fig.canvas.mpl_connect(
            'key_press_event', self.__key_press_callback)
        
    def __key_press_callback(self, event):
        if event.key == 'q':
            print('it worked!')

In [30]:
k = Keybind()

object created


AttributeError: 'Keybind' object has no attribute 'fig'